In [5]:
# Import necessary libraries
import sqlite3
import numpy as np
import matplotlib.pyplot as plt
from pselia.config_elia import settings, load_processed_data_path

/home/yacine/Documents/PhD/Code/GitProject/PBSHM_mdof


In [6]:
database_processed_path = load_processed_data_path('SETTINGS1')
def load_freq_axis(database_processed_path):
    conn = sqlite3.connect(database_processed_path)
    c = conn.cursor()
    c.execute("SELECT freq FROM metadata")
    freq = c.fetchone()[0]
    freq = np.frombuffer(freq, dtype=np.float64)
    return freq

In [7]:
def construct_nodge(freq, f_oi, length, amp):
    hann_len = np.sum(np.abs(freq - f_oi) <= length/2)
    hann = 1 - np.hanning(hann_len) * amp
    window = np.ones(freq.shape)
    window[np.abs(freq - f_oi) <= length/2] = hann
    return window

In [20]:
def add_notch(psd,window):
    if psd.ndim==1:
        psd = psd.reshape(1,-1)
    psd_log = np.log(psd)
    min_psd = np.min(psd_log,axis=-1,keepdims=True)
    max_psd = np.max(psd_log,axis=-1,keepdims=True)
    psd_log_n = (psd_log - min_psd) / (max_psd - min_psd)
    psd_log_n_aff = psd_log_n + (1-window) 
    psd_unnorm_aff = psd_log_n_aff * (max_psd - min_psd) + min_psd
    res = np.exp(psd_unnorm_aff)
    return res


In [21]:
def affect_psd(psd, freq, amp, f_oi, length):
    window = construct_nodge(freq=freq, f_oi=f_oi, length=length, amp=amp)
    psd_res = add_notch(psd, window)
    return psd_res

In [48]:
conn = sqlite3.connect(database_processed_path)
c = conn.cursor()
c.execute("SELECT PSD FROM processed_data WHERE face='2' AND direction='Z' LIMIT 10")
psds = c.fetchall()
psds = np.array([np.frombuffer(psd, dtype=np.float64) for psd, in psds])
freq = load_freq_axis(database_processed_path)


In [62]:
freq_mask = freq < 50
psds[:,freq_mask].shape


TypeError: 'tuple' object is not callable

In [63]:
# Define the layout of the Dash app
import dash
import dash_core_components as dcc
from dash import html
from dash.dependencies import Input, Output
import plotly.graph_objs as go

# Define the app
app = dash.Dash(__name__)

# Define the layout
app.layout = html.Div([
    html.H1('Interactive PSD Plot'),

    html.Label('Amplitude:'),
    dcc.Slider(
        id='amp-slider',
        min=-1,
        max=1.0,
        step=0.01,
        value=0.1
    ),

    html.Label('f_oi:'),
    dcc.Slider(
        id='f-oi-slider',
        min=1,
        max=50,
        step=1,
        value=7
    ),
    html.Label('Length:'),
    dcc.Slider(
        id='length-slider',
        min=1,
        max=50,
        step=1,
        value=1
    ),
    html.Label('PSD_n:'),
    dcc.Slider(
        id='psd-n-slider',
        min=0,
        max=len(psds)-1,
        step=1,
        value=1
    ),

    dcc.Graph(id='psd-plot')
])

# Define the callback function
@app.callback(
    Output('psd-plot', 'figure'),
    Input('amp-slider', 'value'),
    Input('f-oi-slider', 'value'),
    Input('length-slider', 'value'),
    Input('psd-n-slider', 'value')
)
def update_psd_plot(amp, f_oi, length, psd_n):
    # Load the frequency axis
    freq = load_freq_axis(database_processed_path)

    # Create a notch filter with the specified parameters
    notch_filter = construct_nodge(freq, f_oi, length=length, amp=amp)

    # Apply the notch filter to the PSD
    psds_affected = add_notch(psds, notch_filter)

    # Create the Plotly figure
    freq_mask = freq < 50
    trace1 = go.Scatter(x=freq[freq_mask], y=psds[psd_n,freq_mask].T, mode='lines', name='original')
    trace2 = go.Scatter(x=freq[freq_mask], y=psds_affected[psd_n, freq_mask].T, mode='lines', name='affected')
    
    layout = go.Layout(
        title='Interactive PSD Plot',
        xaxis=dict(title='Frequency [Hz]'),
        yaxis=dict(title='PSD [g**2/Hz]', type='log'),
        showlegend=True
    )

    fig = go.Figure(data=[trace1,trace2], layout=layout)
    return fig

# Run the app
app.run_server(mode='inline')